In [1]:
import networkx as nx
import matplotlib.pyplot as plt

In [1]:
import networkx as nx

def print_graph_edges(graph):
    # Print all edges with their weights
    print("Nodes and weights between them:")
    for u, v, data in graph.edges(data=True):
        weight = data.get('weight', 1)  # Default weight to 1 if not provided
        print(f"{u} -- {v} [weight={weight}]")

# Load the graph from a gpickle file
if __name__ == "__main__":
    # Path to the gpickle file
    file_path = input("Enter file name : ")
    
    try:
        # Load the graph from the gpickle file
        graph = nx.read_gpickle(file_path+".gpickle")
        
        # Print all nodes and weights
        print_graph_edges(graph)
    except FileNotFoundError:
        print("The specified gpickle file was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


Enter file name : network_2
Nodes and weights between them:
1 -- 2 [weight=1310]
1 -- 3 [weight=760]
1 -- 4 [weight=390]
2 -- 3 [weight=550]
2 -- 5 [weight=390]
2 -- 8 [weight=450]
3 -- 4 [weight=660]
3 -- 5 [weight=210]
3 -- 6 [weight=390]
4 -- 7 [weight=340]
4 -- 8 [weight=1090]
4 -- 10 [weight=660]
5 -- 6 [weight=220]
5 -- 8 [weight=300]
5 -- 11 [weight=930]
8 -- 6 [weight=400]
8 -- 9 [weight=600]
8 -- 11 [weight=820]
6 -- 7 [weight=730]
6 -- 9 [weight=350]
7 -- 9 [weight=565]
7 -- 10 [weight=320]
10 -- 9 [weight=730]
10 -- 11 [weight=820]
11 -- 9 [weight=320]


In [2]:
import networkx as nx

def sparse_regeneration_algorithm(graph, requests, distance_threshold, Cr, Cm):
    results = []  
    for request in requests:
        source = request['source']
        destination = request['destination']
        bandwidth = request['bandwidth']  
        
        try:   
            # Find the shortest path using Dijkstra's algorithm
            path = nx.dijkstra_path(graph, source=source, target=destination, weight='weight')
            total_distance = sum(graph[path[i]][path[i+1]]['weight'] for i in range(len(path) - 1))
            regenerations = 0
            current_distance = 0
            
            # Calculate regenerations along the path
            for i in range(1, len(path)):
                link_distance = graph[path[i-1]][path[i]]['weight']
                current_distance += link_distance  
                
                # Increment regenerations if the link distance exceeds the threshold
                if link_distance > distance_threshold:
                    regenerations += 1
                    current_distance = 0 
            
            # Calculate regeneration cost
            regeneration_cost = (regenerations * Cr) + (Cm * total_distance)
            
            results.append({
                'source': source,
                'destination': destination,
                'path': path,
                'total_distance': total_distance,
                'regenerations': regenerations,
                'regeneration_cost': regeneration_cost,
                'bandwidth': bandwidth
            })
        except nx.NetworkXNoPath:
            print(f"No path found between {source} and {destination}.")
            results.append({
                'source': source,
                'destination': destination,
                'path': None,
                'total_distance': None,
                'regenerations': None,
                'regeneration_cost': None,
                'bandwidth': bandwidth
            })
    return results

# Main script to load the graph and run the algorithm
if __name__ == "__main__":
    # Input the file path for the gpickle file
    file_path = input("Enter file name : ")
    
    try:
        # Load the graph from the gpickle file
        graph = nx.read_gpickle(file_path+".gpickle")
        
        # Input algorithm parameters
        # Cr=int(input("Enter unit regeneration cost : "))
        # Cm=int(input("Enter cost per unit length : "))
        Cr=75000
        Cm=20000
        distance_threshold = 1000 
        
        # Define requests (modify these as needed or add more)
#         num_requests = int(input("Enter the number of requests: "))
        num_requests=1
        requests = []
        for _ in range(num_requests):
            source = int(input("Enter the source node: "))
            destination = int(input("Enter the destination node: "))
            bandwidth = float(input("Enter the bandwidth: "))
            requests.append({'source': source, 'destination': destination, 'bandwidth': bandwidth})
        
        # Run the sparse regeneration algorithm
        results = sparse_regeneration_algorithm(graph, requests, distance_threshold, Cr, Cm)
        
        # Print results
        for result in results:
            print("\nPath Result:")
            for key, value in result.items():
                print(f"{key}: {value}")
    except FileNotFoundError:
        print("The specified gpickle file was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


Enter file name : network_2
Enter the source node: 1
Enter the destination node: 10
Enter the bandwidth: 50

Path Result:
source: 1
destination: 10
path: [1, 4, 10]
total_distance: 1050
regenerations: 0
regeneration_cost: 21000000
bandwidth: 50.0
